In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from sklearn.linear_model import Lasso
from scipy.io import loadmat
from sklearn.metrics import mean_squared_error
from scipy.integrate import solve_ivp
import os
import sys
sys.path.append('../')  

import pysindy as ps
# Seed the random number generators for reproducibility
np.random.seed(100)

In [ ]:
from test_data import experiment_data,add_noise,normalise_frobenius,normalise_minmax,create_data_2d
from numpy.linalg import matrix_rank, svd
from itertools import product
from utils import plot_result_jacobi
from error_bounds import *

: 

In [ ]:
def get_derivatives(u,dt,dx,fd_order,drop_endpoints=True,remove_endpoints=True):
    #Compute the derivatives
    ut_fd = ps.FiniteDifference(order=fd_order, d=1, axis=1, drop_endpoints=drop_endpoints)._differentiate(u, t=dt)
    utx_fd = ps.FiniteDifference(order=fd_order, d=1, axis=0, drop_endpoints=drop_endpoints)._differentiate(ut_fd, dx)
    ux_fd = ps.FiniteDifference(order=fd_order, d=1, axis=0, drop_endpoints=drop_endpoints)._differentiate(u, dx)
    uxx_fd = ps.FiniteDifference(order=fd_order, d=2, axis=0, drop_endpoints=drop_endpoints)._differentiate(u, dx) 

    if remove_endpoints:
        #Filter out the boundary values
        filter_func=utx_fd
        ut_fd= ut_fd[~np.isnan(filter_func).all(axis=1)][:, ~np.isnan(filter_func).all(axis=0)]
        utx_fd= utx_fd[~np.isnan(filter_func).all(axis=1)][:, ~np.isnan(filter_func).all(axis=0)]
        ux_fd= ux_fd[~np.isnan(filter_func).all(axis=1)][:, ~np.isnan(filter_func).all(axis=0)]
        uxx_fd= uxx_fd[~np.isnan(filter_func).all(axis=1)][:, ~np.isnan(filter_func).all(axis=0)]
    return ut_fd, utx_fd,ux_fd, uxx_fd

def drop_endpoints(func_fd,func_true,filter_func):
    func_true = func_true[~np.isnan(filter_func).all(axis=1)][:, ~np.isnan(filter_func).all(axis=0)]
    func_fd= func_fd[~np.isnan(filter_func).all(axis=1)][:, ~np.isnan(filter_func).all(axis=0)]
    return func_fd,func_true

: 

We want to get a threshold and error bound for analysing a analytic or polynomial PDE as unique. We can reuse the functions from the SFranco method.

## Easy examples: $u_1 = x^2 + t^2$ and $u_2 = exp(x-at)$

In [ ]:
n_samples=50
T,X,t,x = create_data_2d(T_start=-2, T_end=2, L_x_start=0,L_x_end=5, N_t=n_samples, N_x=n_samples)
a=np.random.randn()
"""
#not unique
u = X**2 + T**2
ux = 2*X
ut = 2*T
utx = 0*T
uxx = 2*np.ones(X.shape)
"""

#not unqiue
u = np.exp(X-a*T)
ux = u
ut = -a*u
utx = -a*u
uxx = u
"""
#analytic unique
u=(X + T)*np.arccos(1/np.cosh(a*T))
ut = a*(T + X)*np.sinh(T*a)/(np.sqrt(1 - 1/np.cosh(T*a)**2)*np.cosh(T*a)**2) + np.arccos(1/np.cosh(T*a))
ux = np.arccos(1/np.cosh(T*a))
utx = a*np.sinh(T*a)/(np.sqrt(1 - 1/np.cosh(T*a)**2)*np.cosh(T*a)**2)
uxx = np.zeros(X.shape)
"""
print(f"a {a}")
dt=t[1]-t[0]
dx=x[1]-x[0]

: 

#### Compare the error on finite differences on the Jacobian of $g = ( u | u_x)$

In [ ]:
#Compute the derivatives
fd_order=10
ut_fd, utx_fd,ux_fd, uxx_fd = get_derivatives(u,dt,dx,fd_order,drop_endpoints=True,remove_endpoints=False)

#Filter out the boundary values
filter_func = utx_fd
ut_fd,ut = drop_endpoints(ut_fd,ut,filter_func)
utx_fd,utx = drop_endpoints(utx_fd,utx,filter_func)
ux_fd,ux = drop_endpoints(ux_fd,ux,filter_func)
uxx_fd,uxx = drop_endpoints(uxx_fd,uxx,filter_func)

: 

##### Compare the upper bounds for different derivatives and the Jacobian

###### Without noise

Lets calculate an upper bound for the different derivatives

In [ ]:
eps=1e-15

bound_ut = upper_bound_central_differences(eps,fd_order,order_derivative=1,Cu=infinity_norm(a**(fd_order+1)*u),Cxi=1.0,h=dt)
bound_ux = upper_bound_central_differences(eps,fd_order,order_derivative=1,Cu=infinity_norm(u),Cxi=1.0,h=dx)
bound_uxx = upper_bound_central_differences(eps,fd_order,order_derivative=2,Cu=infinity_norm(u),Cxi=1.0,h=dx)
bound_utx = upper_bound_central_differences(bound_ut,fd_order,order_derivative=1,Cu=infinity_norm(ut),Cxi=1.0,h=dx)

print(f" |ut_fd - ut| \t= {infinity_norm((ut_fd-ut).flatten()):.3e} <= {bound_ut:.3e}")
print(f" |ux_fd - ux| \t= {infinity_norm((ux_fd-ux).flatten()):.3e} <= {bound_ux:.3e}")
print(f" |uxx_fd - uxx|\t= {infinity_norm((uxx_fd-uxx).flatten()):.3e} <= {bound_uxx:.3e}")
print(f" |utx_fd - utx|\t= {infinity_norm((utx_fd-utx).flatten()):.3e} <= {bound_utx:.3e}")

: 

In [ ]:
space_range = int(ut.shape[0]/10-1)
time_range = int(ut.shape[1]/10-1)
jacobians =[]
upperbounds=[]
for i in range(space_range):
    for j in range(time_range):
        x_i, t_j = i * 10 + 10, j * 10 + 10
        jacobian_fd = np.array([[ut_fd[x_i,t_j], ux_fd[x_i,t_j]], [utx_fd[x_i,t_j], uxx_fd[x_i,t_j]]]).reshape(2,2)
        jacobian = np.array([[ut[x_i,t_j], ux[x_i,t_j]], [utx[x_i,t_j], uxx[x_i,t_j]]]).reshape(2,2)

        jacobian_diff = np.linalg.norm(jacobian_fd -jacobian)
        nr_datapoints=1
        upperbound = np.sqrt(bound_ut**2+bound_ux**2 + bound_uxx**2+bound_utx**2)

        jacobians.append(jacobian_diff)
        upperbounds.append(upperbound)

        print("Check wether upper bound is correct:")
        print(f"|JG-JG_fd|= {jacobian_diff:.3e} <= {upperbound:.3e}")

plt.plot(jacobians,label="|JG-JG_fd|")
plt.plot(upperbounds, label="upper bound")
plt.xlabel("Datapoints")
plt.ylabel("Value")
plt.legend()
plt.show()

: 

In [ ]:
M=infinity_norm(u)
number_noise=10 #<=10
noise_levels = [0]+[10**(-10+i) for i in range(0,number_noise-1)]
orders=range(2,10,2)
unorm2=np.sqrt(np.mean(np.square(u)))

for noise_level in noise_levels: 
    var = noise_level * unorm2
    noise = np.random.normal(0, var, size=u.shape)
    u_noise = u + noise
    print(f"Noise level {noise_level}")
    #Calculate upper bounds
    eps = infinity_norm(u-u_noise)
    bound_ut = upper_bound_central_differences(eps,fd_order,order_derivative=1,Cu=infinity_norm(a**(fd_order+1)*u),Cxi=1.0,h=dt)
    bound_ux = upper_bound_central_differences(eps,fd_order,order_derivative=1,Cu=infinity_norm(u),Cxi=1.0,h=dx)
    bound_uxx = upper_bound_central_differences(eps,fd_order,order_derivative=2,Cu=infinity_norm(u),Cxi=1.0,h=dx)
    bound_utx = upper_bound_central_differences(bound_ut,fd_order,order_derivative=1,Cu=infinity_norm(ut),Cxi=1.0,h=dx)
    
    #Calculate derivatives
    ut_fd, utx_fd,ux_fd, uxx_fd= get_derivatives(u_noise,dt,dx,fd_order)
    
    print(f" |ut_fd - ut| \t= {infinity_norm((ut_fd-ut).flatten()):.3e} <= {bound_ut:.3e}")
    print(f" |ux_fd - ux| \t= {infinity_norm((ux_fd-ux).flatten()):.3e} <= {bound_ux:.3e}")
    print(f" |uxx_fd - uxx|\t= {infinity_norm((uxx_fd-uxx).flatten()):.3e} <= {bound_uxx:.3e}")
    print(f" |utx_fd - utx|\t= {infinity_norm((utx_fd-utx).flatten()):.3e} <= {bound_utx:.3e}")
    
    jacobians =[]
    #Calculate upper bound which stays constant for all data points(only changes in noise level)
    nr_datapoints=1
    upperbound = np.sqrt(bound_ut**2+bound_ux**2 + bound_uxx**2+bound_utx**2)
    print(f"Upper bound {upperbound:.3e}")
    for i in range(space_range):
        for j in range(time_range):
            #Compute differenc between true Jacobian and finite difference Jacobian
            x_i, t_j = i * 10 + 10, j * 10 + 10
            jacobian_fd = np.array([[ut_fd[x_i,t_j], ux_fd[x_i,t_j]], [utx_fd[x_i,t_j], uxx_fd[x_i,t_j]]]).reshape(2,2)
            jacobian = np.array([[ut[x_i,t_j], ux[x_i,t_j]], [utx[x_i,t_j], uxx[x_i,t_j]]]).reshape(2,2)
            jacobian_diff = np.linalg.norm(jacobian_fd -jacobian)    
            jacobians.append(jacobian_diff)
    
            #print("Check wether upper bound is correct:")
            #print(f"|JG-JG_fd|= {jacobian_diff:.3e} <= {upperbound:.3e}")
    plt.title(f"Noise level {noise_level}, Order {fd_order}")
    plt.plot(jacobians,label="|JG-JG_fd|")
    plt.axhline(y=upperbound, color='r', linestyle='-',label="upperbound")
    plt.xlabel("Datapoints")
    plt.ylabel("Value")
    plt.legend()
    filename = f"Images/noise_{noise_level}_order_{fd_order}.png"
    plt.savefig(filename)
    plt.show()

: 

Compare Jacobians and singular values

In [ ]:
def get_upper_bound_jacobian(eps,fd_order,Cut,Cux,Cuxx,Cutx,dt,dx,Cxi=1.0):

    bound_ut = upper_bound_central_differences(eps,fd_order,order_derivative=1,Cu=Cut,Cxi=Cxi,h=dt)
    bound_ux = upper_bound_central_differences(eps,fd_order,order_derivative=1,Cu=Cux,Cxi=Cxi,h=dx)
    bound_uxx = upper_bound_central_differences(eps,fd_order,order_derivative=2,Cu=Cuxx,Cxi=Cxi,h=dx)
    bound_utx = upper_bound_central_differences(bound_ut,fd_order,order_derivative=1,Cu=Cutx,Cxi=Cxi,h=dx)

    upper_bound = np.sqrt(bound_ut**2+bound_ux**2 + bound_uxx**2+bound_utx**2)
    
    return upper_bound

: 

In [ ]:
space_range = int(ut.shape[1]/10-2)
time_range = int(ut.shape[0]/10-2)

#For saving results
svs = np.zeros([space_range, time_range])
svs_fd = np.zeros([space_range, time_range])
upper_bounds = np.zeros([space_range, time_range])
lower_bounds = np.zeros([space_range, time_range])

ut_fd, utx_fd,ux_fd, uxx_fd = get_derivatives(u,dt,dx,fd_order)

for i in range(space_range):
    for j in range(time_range):
        x_i, t_j = i * 10 + 10, j * 10 + 10
        jacobian_fd = np.array([[ut_fd[x_i,t_j], ux_fd[x_i,t_j]], [utx_fd[x_i,t_j], uxx_fd[x_i,t_j]]]).reshape(2,2)
        jacobian = np.array([[ut[x_i,t_j], ux[x_i,t_j]], [utx[x_i,t_j], uxx[x_i,t_j]]]).reshape(2,2)
        

        sv = svd(jacobian, compute_uv=False)
        sv_fd = svd(jacobian_fd, compute_uv=False)
        sv_min,sv_min_fd =sv  [-1],sv_fd[-1]
        sv_max,sv_max_fd =sv[0],sv_fd[0]

        #Save the results 
        svs[i,j] = sv_min/sv_max
        svs_fd[i,j] =sv_min_fd/sv_max_fd
        jacobian_diff=np.linalg.norm(jacobian_fd -jacobian)
        upper_bound_jacobian = get_upper_bound_jacobian(1e-16,fd_order,infinity_norm(a**(fd_order+1)*u),
                                                        infinity_norm(u),infinity_norm(u),infinity_norm(ut),dt,dx)
        print("Jacobian diff ", jacobian_diff, "upper_bound_jacobian",upper_bound_jacobian)
        
        C1=sv_max*1.5 #C1 => o_max
        C=sv_max*0.5 #C <= o_max
        C2=1e-4
        lower_bound = lower_bound_nonsingular_matrix(C1,C2,jacobian_diff)
        upper_bound = upper_bound_singular_matrix(C,jacobian_diff)

        lower_bound_nonexact = lower_bound_nonsingular_matrix(C1,C2,upper_bound_jacobian)
        upper_bound_nonexact = upper_bound_singular_matrix(C,upper_bound_jacobian)

        lower_bounds[i,j]=lower_bound
        upper_bounds[i,j]=upper_bound
        
        print(f" Comparison at x = {x_i}, t = {t_j} ")
        print(f" \t \t \t \t Original \t FD \t \t diff")
        print(f" Smallest singular values \t {sv_min:.3e}\t  {sv_min_fd:.3e} \t {np.abs(sv_min-sv_min_fd):.3e}")
        print(f" Biggest singular values \t {sv_max:.3e}\t  {sv_max_fd:.3e} \t {np.abs(sv_max-sv_max_fd):.3e}")
        print(f" Differences Jacobian \t \t {jacobian_diff:.3e} <= {upper_bound_jacobian:.3e}")
        print(" Bounds with jacobian difference")
        print(f" \t Lower bound for uniqueness: \t  {svs_fd[i,j]:.3e} >= {lower_bound:.3e}")
        print(f" \t Upper bound for non uniqueness:  {svs_fd[i,j]:.3e} <= {upper_bound:.3e}")
        print(" Bounds with upper bound on jacobian difference")
        print(f" \t Lower bound for uniqueness: \t  {svs_fd[i,j]:.3e} >= {lower_bound_nonexact:.3e}")
        print(f" \t Upper bound for non uniqueness:  {svs_fd[i,j]:.3e} <= {upper_bound_nonexact:.3e}")
        print(f" True ratio: \t  \t \t  {svs[i,j]:.3e} ")

: 

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
green=(100/255,250/255,100/255)#(0,1,0)
red=(241/255,13/255,30/255)
cmap_green_red = LinearSegmentedColormap.from_list('RedGreen', [green,red], N=256)
cmap_red_green = LinearSegmentedColormap.from_list('GreenRed', [red,green], N=256)

: 

In [ ]:
lower_bounds

: 

In [ ]:
upper_bounds

: 

In [ ]:
svs_flat = svs.flatten()
lower_bounds_flat = lower_bounds.flatten()
upper_bounds_flat = upper_bounds.flatten()

# Get the maximum value among the maximum values of all three arrays


: 

In [ ]:
def plot_ratio_upper_lower_old(svs,lower_bounds,upper_bounds,title=""):
    fig, ax = plt.subplots(1, 1, sharex=True, figsize=(6, 4))
    n=len(svs.flatten())
    ax.set_title(title)

    max_value = np.maximum.reduce([svs.flatten().max(), lower_bounds.flatten().max(), upper_bounds_flat.flatten().max()])

    ax.plot( svs.flatten(),color='blue')
    ax.fill_between(np.arange(n),lower_bounds.flatten(), max_value,color='coral', alpha=0.3)
    ax.plot(lower_bounds.flatten(),label="unique",color='coral')
    ax.set_ylabel('Threshold/Ratio')


    ax.fill_between(np.arange(n),upper_bounds.flatten(), y2=0, color='aquamarine', alpha=0.3)
    ax.set_yscale('log')
    ax.plot(upper_bounds.flatten(),label="not unique", color='aquamarine')

    ax.set_xlabel('Datapoints')
    ax.legend(loc=4)

: 

In [ ]:
# Create subplots
plot_ratio_upper_lower_old(svs,lower_bounds,upper_bounds,f"Noise level {0}, Order {fd_order}")

: 

In [ ]:
t_grid, x_grid = (np.arange(time_range) * 10 + 10) / len(t) * (t[len(t)-1] - t[0]) + t[0], (np.arange(space_range) * 10 + 10) / len(x) * (x[len(x)-1] - x[0]) + x[0]
# Plot derivative results
subtitle_size=16
tickssize=16
order=fd_order
fig, axes = plt.subplots(1,3, figsize=(12,4))
plt.suptitle(f'Noise level 0', fontsize=subtitle_size,y=1.1)

#Plot the ratios
c = axes[0].pcolor(t_grid, x_grid, svs)
axes[0].set_title('Ratio', fontsize=tickssize)
axes[0].set_ylabel('x', fontsize=tickssize)
axes[0].set_xlabel('t', fontsize=tickssize)
fig.colorbar(c, ax=axes[0])

upper_minus_svs = upper_bounds - svs # <= 0 if unique >= 0 if non-unique
lower_minus_svs = lower_bounds - svs # <= 0 if unique >= 0 if non-unique
max_svs= np.max(np.abs(upper_minus_svs))
#Plot difference to upper bound
axes[1].set_title('Upper bound - ratio', fontsize=tickssize)
c=axes[1].pcolor(t_grid, x_grid, upper_minus_svs,cmap=cmap_red_green,vmin=-max_svs, vmax=max_svs)
axes[1].set_yticks([])
fig.colorbar(c, ax=axes[1])

#Plot difference to lower bound
max_svs= np.max(np.abs(lower_minus_svs))
axes[2].set_title('Lower bound - ratio', fontsize=tickssize)
c = axes[2].pcolor(t_grid, x_grid, lower_minus_svs, cmap=cmap_red_green, vmin=-max_svs, vmax=max_svs)
axes[2].set_yticks([])
cb = fig.colorbar(c, ax=axes[2])
ticks=np.linspace(-max_svs, max_svs,7) 
cb.set_ticks(ticks)
cb.set_ticklabels(['Unique', f'{ticks[1]:.4f}',f'{ticks[2]:.4f}',f'{ticks[3]:.4f}', f'{ticks[4]:.4f}', f'{ticks[5]:.4f}', 'Non-unique'])

plt.tight_layout()

: 

### Test different noise levels on easy examples

In [ ]:
def plot_results(upper_bounds, lower_bounds,svs,time_range,space_range,subtitle="",subtitle_size=16,tickssize=16):
    t_grid, x_grid = (np.arange(time_range) * 10 + 10) / len(t) * (t[len(t)-1] - t[0]) + t[0], (np.arange(space_range) * 10 + 10) / len(x) * (x[len(x)-1] - x[0]) + x[0]
    fig, axes = plt.subplots(1,3, figsize=(12,4))
    #Plot the ratios
    c = axes[0].pcolor(t_grid, x_grid, svs)
    axes[0].set_title('Ratio', fontsize=tickssize)
    axes[0].set_ylabel('x', fontsize=tickssize)
    axes[0].set_xlabel('t', fontsize=tickssize)
    fig.colorbar(c, ax=axes[0])
    
    uupper_minus_svs = upper_bounds - svs # <= 0 if unique >= 0 if non-unique
    lower_minus_svs = lower_bounds - svs # <= 0 if unique >= 0 if non-unique
    max_svs= np.max(np.abs(upper_minus_svs))
    #Plot difference to upper bound
    plt.suptitle(subtitle, fontsize=subtitle_size,y=1.1)
    
    axes[1].set_title('Upper bound - ratio', fontsize=tickssize)
    c=axes[1].pcolor(t_grid, x_grid, upper_minus_svs,cmap=cmap_red_green,vmin=-max_svs, vmax=max_svs)
    axes[1].set_yticks
    fig.colorbar(c, ax=axes[1])
    
    #Plot difference to lower bound
    max_svs= np.max(np.abs(lower_minus_svs))
    axes[2].set_title('Lower bound - ratio', fontsize=tickssize)
    c = axes[2].pcolor(t_grid, x_grid, lower_minus_svs, cmap=cmap_red_green, vmin=-max_svs, vmax=max_svs)
    axes[2].set_yticks([])
    cb = fig.colorbar(c, ax=axes[2])
    ticks=np.linspace(-max_svs, max_svs,7) 
    cb.set_ticks(ticks)
    cb.set_ticklabels(['Unique', f'{ticks[1]:.4f}',f'{ticks[2]:.4f}',f'{ticks[3]:.4f}', f'{ticks[4]:.4f}', f'{ticks[5]:.4f}', 'Non-unique'])
    #plt.savefig(f"Images/JRC_{subtitle}")
    plt.tight_layout()


: 

In [ ]:
def get_results(space_range,time_range,u,fd_order,dt,dx,eps):
    
    ut_fd, utx_fd,ux_fd, uxx_fd= get_derivatives(u,dt,dx,fd_order)
    #For saving results
    svs = np.zeros([space_range, time_range])
    upper_bounds = np.zeros([space_range, time_range])
    lower_bounds = np.zeros([space_range, time_range])  
    
    for i in range(space_range):
        for j in range(time_range):
            x_i, t_j = i * 10 + 10, j * 10 + 10
            jacobian_fd = np.array([[ut_fd[x_i,t_j], ux_fd[x_i,t_j]], [utx_fd[x_i,t_j], uxx_fd[x_i,t_j]]]).reshape(2,2)

            sv_fd = svd(jacobian_fd, compute_uv=False)
            sv_min =sv_fd[-1]
            sv_max =sv_fd[0]
    
            #Save the results 
            svs[i,j] = sv_min/sv_max
            C1=sv_max*1.5
            C=sv_max*0.5
            C2=max(1e-3,sv_min*0.5)
            #print("C2 :", C2)

            upper_bound_jacobian = get_upper_bound_jacobian(eps,fd_order,infinity_norm(a**(fd_order+1)*u),
                                                infinity_norm(u),infinity_norm(u),infinity_norm(ut),dt,dx)

            #Use the real difference between jacobian and jacobian_fd
            #jacobian = np.array([[ut[x_i,t_j], ux[x_i,t_j]], [ut[x_i,t_j], ux[x_i,t_j]]]).reshape(2,2)
            #upper_bound_jacobian = np.linalg.norm(jacobian-jacobian_fd)
            
            lower_bound = lower_bound_nonsingular_matrix(C1,C2,upper_bound_jacobian)
            upper_bound = upper_bound_singular_matrix(C,upper_bound_jacobian)

            lower_bounds[i,j]=lower_bound
            upper_bounds[i,j]=upper_bound
    return svs, lower_bounds,upper_bounds

: 

In [ ]:
M=infinity_norm(u)
number_noise=10 #<=10
#noise_levels = [0]+[10**(-10+i) for i in range(0,number_noise-1)]
noise_levels = [0,1e-08,1e-05,1e-02]
orders=range(2,10,2)
unorm2=np.sqrt(np.mean(np.square(u)))

for noise_level in noise_levels:    
    
    var = noise_level * unorm2
    noise = np.random.normal(0, var, size=u.shape)
    u_noise = u + noise
    eps = infinity_norm(u-u_noise)
    
    print(f"Noise level {noise_level}")
    fd_orders=(2,8)
    svs, lower_bounds,upper_bounds = get_results(space_range,time_range,u_noise,fd_orders[0],dt,dx,eps)
    plot_results(upper_bounds, lower_bounds,svs,time_range,space_range,subtitle=f"Noise level {noise_level}, Order {fd_orders[0]}")
    svs, lower_bounds,upper_bounds = get_results(space_range,time_range,u_noise,fd_orders[1],dt,dx,eps)
    plot_results(upper_bounds, lower_bounds,svs,time_range,space_range,subtitle=f"Noise level {noise_level}, Order {fd_orders[1]}")

: 

In [ ]:
for val_svs,val_lb, val_ub in zip(svs,lower_bounds, upper_bounds):
    print(val_svs, " \t ", val_lb, " \t  ", val_ub)

: 

In [ ]:
svs

: 

In [ ]:
M=infinity_norm(u)
number_noise=10 #<=10
#noise_levels = [0]+[10**(-10+i) for i in range(0,number_noise-1)]
noise_levels = [0,1e-08,1e-05,1e-02]
orders=range(2,10,2)
unorm2=np.sqrt(np.mean(np.square(u)))

for noise_level in noise_levels:    
    
    var = noise_level * unorm2
    noise = np.random.normal(0, var, size=u.shape)
    u_noise = u + noise
    eps = infinity_norm(u-u_noise)
    
    print(f"Noise level {noise_level}")
    fd_orders=(2,8)
    svs, lower_bounds,upper_bounds = get_results(space_range,time_range,u_noise,fd_orders[0],dt,dx,eps)
    print(f"Order {fd_orders[0]}")
    print(" \t omax/omin \t \t \t  lower_bounds \t \t upper_bounds")
    #Print Values
    for val_svs,val_lb, val_ub in zip(svs,lower_bounds, upper_bounds):
        print(" \t ",val_svs, " \t ", val_lb, " \t ", val_ub)
    #svs, lower_bounds,upper_bounds = get_results(space_range,time_range,u_noise,fd_orders[1],dt,dx,eps)

: 

In [ ]:
M=infinity_norm(u)
number_noise=10 #<=10
#noise_levels = [0]+[10**(-10+i) for i in range(0,number_noise-1)]
noise_levels = [0,1e-08,1e-05,1e-02]
orders=range(2,10,2)
unorm2=np.sqrt(np.mean(np.square(u)))

for noise_level in noise_levels:    
    
    var = noise_level * unorm2
    noise = np.random.normal(0, var, size=u.shape)
    u_noise = u + noise
    eps = infinity_norm(u-u_noise)
    
    print(f"Noise level {noise_level}")
    fd_orders=(2,8)
    
    fig, axes = plt.subplots(1, 2, sharex=True, figsize=(8, 4))
    n=len(svs.flatten())
    plt.suptitle(f"Noise level {noise_level}")

    max_value = np.maximum.reduce([svs.flatten().max(), lower_bounds.flatten().max(), upper_bounds_flat.flatten().max()])
    #First Order
    svs, lower_bounds,upper_bounds = get_results(space_range,time_range,u_noise,fd_orders[0],dt,dx,eps)
    axes[0].set_title(f"Order {fd_orders[0]}")
    axes[0].plot( svs.flatten(),color='blue')
    axes[0].fill_between(np.arange(n),lower_bounds.flatten(), max_value,color='coral', alpha=0.3)
    axes[0].plot(lower_bounds.flatten(),label="unique",color='coral')
    axes[0].set_ylabel('Threshold/Ratio')

    axes[0].fill_between(np.arange(n),upper_bounds.flatten(), y2=0, color='aquamarine', alpha=0.3)
    axes[0].set_yscale('log')
    axes[0].plot(upper_bounds.flatten(),label="not unique", color='aquamarine')

    axes[0].set_xlabel('Datapoints')

    #Second order
    svs, lower_bounds,upper_bounds = get_results(space_range,time_range,u_noise,fd_orders[1],dt,dx,eps)
    axes[1].set_title(f"Order {fd_orders[1]}")
    axes[1].plot( svs.flatten(),color='blue',label = r"$\sigma_{min}/\sigma_{max}$")
    axes[1].fill_between(np.arange(n),lower_bounds.flatten(), max_value,color='coral', alpha=0.3)
    axes[1].plot(lower_bounds.flatten(),label="unique",color='coral')

    axes[1].fill_between(np.arange(n),upper_bounds.flatten(), y2=0, color='aquamarine', alpha=0.3)
    axes[1].set_yscale('log')
    axes[1].plot(upper_bounds.flatten(),label="not unique", color='aquamarine')

    axes[1].legend(loc=4)
    

: 

## Try it on different examples

In [ ]:
experiment_names= ['algebraic_nonunique_kdv','algebraic_nonunique_1','analytic_unique_1','analytic_unique_2','analytic_nonunique_1']

: 

### Perform on one experiment

In [ ]:
noise_levels = [0,1e-08,1e-05,0.01]
fd_orders=(2,8)
experiment_name = experiment_names[3]
#Get data
u,x,t,formula = experiment_data(n_samples=50,experiment_name=experiment_name)
dx=x[1]-x[0]
dt=t[1]-t[0]
space_range = int(len(x)/10-1)
time_range = int(len(x)/10-1)
print(experiment_name)
print(formula)
subtitle=f"Experiment {experiment_name} {formula}"

t_grid, x_grid = (np.arange(time_range) * 10 + 10) / len(t) * (t[len(t)-1] - t[0]) + t[0], (np.arange(space_range) * 10 + 10) / len(x) * (x[len(x)-1] - x[0]) + x[0]
fig, axes = plt.subplots(2,6, figsize=(24,8))

for i,noise_level in enumerate(noise_levels):
    u_noise = add_noise(u,noise_level)
    eps = infinity_norm(u-u_noise)
    
    svs, lower_bounds,upper_bounds = get_results(space_range,time_range,u_noise,fd_order,dt,dx,eps)
 
    #Plot the ratios

    c = axes[i//2,i*3%6+0].pcolor(t_grid, x_grid, svs)
    axes[i//2,i*3%6+0].set_title('Ratio', fontsize=tickssize)
    axes[i//2,i*3%6+0].set_ylabel('x', fontsize=tickssize)
    axes[i//2,i*3%6+0].set_xlabel('t', fontsize=tickssize)
    fig.colorbar(c, ax=axes[i//2,i*3%6])
    
    upper_minus_svs = upper_bounds - svs
    lower_minus_svs = lower_bounds -svs
    max_svs= np.max(np.abs(upper_minus_svs))
    #Plot difference to upper bound
    plt.suptitle(subtitle, fontsize=subtitle_size,y=1.1)
 
    
    axes[i//2,i*3%6+1].set_title(f'Noise level {noise_level} \n Upper bound - ratio', fontsize=tickssize)
    c=axes[i//2,i*3%6+1].pcolor(t_grid, x_grid, upper_minus_svs,cmap=cmap_red_green,vmin=-max_svs, vmax=max_svs)
    axes[i//2,i*3%6+1].set_yticks([])
    fig.colorbar(c, ax=axes[i//2,i*3%6+1])
    
   
    #Plot difference to lower bound
    lower_minus_svs = lower_bounds -svs
    max_svs= np.max(np.abs(lower_minus_svs))
    axes[i//2,i*3%6+2].set_title('Lower bound - ratio', fontsize=tickssize)
    c = axes[i//2,i*3%6+2].pcolor(t_grid, x_grid, lower_minus_svs, cmap=cmap_red_green, vmin=-max_svs, vmax=max_svs)
    axes[i//2,i*3%6+2].set_yticks([])
    cb = fig.colorbar(c, ax=axes[i//2,i*3%6+2])
    ticks=np.linspace(-max_svs, max_svs,7) 
    cb.set_ticks(ticks)
    cb.set_ticklabels(['Non-unique', f'{ticks[1]:.4f}',f'{ticks[2]:.4f}',f'{ticks[3]:.4f}', f'{ticks[4]:.4f}', f'{ticks[5]:.4f}', 'Unique'])
    #plt.savefig(f"Images/JRC_{subtitle}")
    plt.tight_layout()


: 

### Perform on all experiments

In [ ]:
def perform_experiment(noise_levels,fd_order,experiment_name):
    
    #Get data,
    u,x,t,formula = experiment_data(n_samples=50,experiment_name=experiment_name)
    dx=x[1]-x[0]
    dt=t[1]-t[0]
    space_range = int(len(x)/10-1)
    time_range = int(len(x)/10-1)
    print(f"Performing experiment {experiment_name} {formula} with order {fd_order}")
    subtitle=f"Experiment {experiment_name} {formula}, Order {fd_order}"
    
    t_grid, x_grid = (np.arange(time_range) * 10 + 10) / len(t) * (t[len(t)-1] - t[0]) + t[0], (np.arange(space_range) * 10 + 10) / len(x) * (x[len(x)-1] - x[0]) + x[0]
    fig, axes = plt.subplots(2,6, figsize=(24,8))
    
    for i,noise_level in enumerate(noise_levels):
        u_noise = add_noise(u,noise_level)
        eps = infinity_norm(u-u_noise)
        
        svs, lower_bounds,upper_bounds = get_results(space_range,time_range,u_noise,fd_order,dt,dx,eps)
     
        #Plot the ratios
    
        c = axes[i//2,i*3%6+0].pcolor(t_grid, x_grid, svs)
        axes[i//2,i*3%6+0].set_title('Ratio', fontsize=tickssize)
        axes[i//2,i*3%6+0].set_ylabel('x', fontsize=tickssize)
        axes[i//2,i*3%6+0].set_xlabel('t', fontsize=tickssize)
        fig.colorbar(c, ax=axes[i//2,i*3%6])
        
        upper_minus_svs = upper_bounds - svs
        lower_minus_svs = lower_bounds -svs
        max_svs= np.max(np.abs(upper_minus_svs))
        #Plot difference to upper bound
        plt.suptitle(subtitle, fontsize=subtitle_size,y=1.1)
     
        
        axes[i//2,i*3%6+1].set_title(f'Noise level {noise_level} \n Upper bound - ratio', fontsize=tickssize)
        c=axes[i//2,i*3%6+1].pcolor(t_grid, x_grid, upper_minus_svs,cmap=cmap_red_green,vmin=-max_svs, vmax=max_svs)
        axes[i//2,i*3%6+1].set_yticks([])
        fig.colorbar(c, ax=axes[i//2,i*3%6+1])
        
       
        #Plot difference to lower bound
        lower_minus_svs = lower_bounds  -svs
        max_svs= np.max(np.abs(lower_minus_svs))
        axes[i//2,i*3%6+2].set_title('Lower bound - ratio', fontsize=tickssize)
        c = axes[i//2,i*3%6+2].pcolor(t_grid, x_grid, lower_minus_svs, cmap=cmap_red_green, vmin=-max_svs, vmax=max_svs)
        axes[i//2,i*3%6+2].set_yticks([])
        cb = fig.colorbar(c, ax=axes[i//2,i*3%6+2])
        ticks=np.linspace(-max_svs, max_svs,7) 
        cb.set_ticks(ticks)
        cb.set_ticklabels(['Non-unique', f'{ticks[1]:.4f}',f'{ticks[2]:.4f}',f'{ticks[3]:.4f}', f'{ticks[4]:.4f}', f'{ticks[5]:.4f}', 'Unique'])
        #plt.savefig(f"Images/JRC_{subtitle}")
        plt.tight_layout()


: 

In [ ]:
noise_levels = [0,1e-08,1e-05,0.01]
fd_orders=[2,8]
for experiment_name in experiment_names:
    for order in fd_orders:
        perform_experiment(noise_levels,order,experiment_name)

: 

: 